<a href="https://colab.research.google.com/github/Shakthivel-K/oops_proj/blob/main/Fall_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Download TFRECORD data
import json

kaggle = {"username":"manikandan02","key":"64d4551e3314adcd54c4f8043eab700a"}

with open('kaggle.json', 'w') as outfile:
  json.dump(kaggle,outfile)

!pip install --upgrade --force-reinstall --no-deps kaggle
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d manikandan02/falldataset
!unzip /content/falldataset.zip


     |████████████████████████████████| 58 kB 5.4 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=099b3cda0c5afb760da521c0d0bd094d17b94e732ff8ddc2c950d769246ec807
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
kaggle.json
100% 12.1G/12.1G [01:43<00:00, 177MB/s]
100% 12.1G/12.1G [01:43<00:00, 125MB/s]
Archive:  /content/falldataset.zip
  inflating: test.tfrecords          
  inflating: train.tfrecords         
  inflating: validation.tfrecords    


## Initialization

In [2]:
import os
import io
import time
import shutil
import random
from pathlib import Path
from tqdm.notebook import tqdm
from zipfile import ZipFile
import urllib.request as urlreq

import imageio
import ipywidgets
from sklearn.model_selection import train_test_split

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.compat.v1.keras import backend as k
from tensorflow.keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,Dense,Dropout
from tensorflow.keras.metrics import Precision, Recall


In [3]:

#Set seed value
seed_value = 43

os.environ['PYTHONHASHSEED'] = str(seed_value)

random.seed(seed_value)

#numpy seed
np.random.seed(seed_value)

#Tf seed
tf.random.set_seed(seed_value)

#Configure new global tensorflow session

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads = 1
)

sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
k.set_session(sess)

## Hyperparameters

In [5]:
cfg = {
        'frameCount'      :50,
        'dataset'         :'CMDFALL & URFD',
        'epochs'          :300,
        'seed_value'      :43,
        'train_split'     :0.7,
        'val_split'       :0.2,
        'test_split'      :0.1,
        'batch_size'      :4,
        'optimizer'       :'adam',
        'INPUT_SHAPE'     : (50,224,224, 1),
        'NUM_CLASSES'     :2,
        'LEARNING_RATE'   :1e-4,
        'WEIGHT_DECAY'    :1e-5,
        'PATCH_SIZE'      :(1,224,224),
        'NUM_PATCHES'     : 50, #(50//10)**2,
        'LAYER_NORM_EPS'  : 1e-6,
        'PROJECTION_DIM'  : 32,
        'NUM_HEADS'       : 2,
        'NUM_LAYERS'      : 2
       }

## Datapipeline

In [6]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label

In [7]:
def parser(record):
  keys_to_feature = {
      'image_raw' :tf.io.FixedLenFeature([],tf.string),
      'label'     : tf.io.FixedLenFeature([],tf.int64)
  }

  parsed = tf.io.parse_single_example(record, keys_to_feature)
  image = tf.io.decode_raw(parsed['image_raw'], tf.uint8)
  image = tf.reshape(image, shape=[50,224,224])
  image = tf.image.convert_image_dtype(image,tf.float32,)
  label = tf.cast(parsed['label'], tf.int16)
  return image,label

trainloader = tf.data.TFRecordDataset(filenames='train.tfrecords').map(parser, num_parallel_calls=tf.data.AUTOTUNE).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(cfg['batch_size']).prefetch(tf.data.AUTOTUNE)
validloader = tf.data.TFRecordDataset(filenames='validation.tfrecords').map(parser, num_parallel_calls=tf.data.AUTOTUNE).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(cfg['batch_size']).prefetch(tf.data.AUTOTUNE)
testloader = tf.data.TFRecordDataset(filenames='test.tfrecords').map(parser, num_parallel_calls=tf.data.AUTOTUNE).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(cfg['batch_size']).prefetch(tf.data.AUTOTUNE)


## Model

Referenced for Keras examples

### Tubelet Embedding

In [8]:

class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.patch_size = patch_size
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",    
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def get_config(self):
        config = super().get_config()
        config.update({
            'patch_size' : self.patch_size,
            'embed_dim'  : self.embed_dim
        })
        return config

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches


### Positional encoding

In [12]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim'  : self.embed_dim
        })
        return config

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens


### ViViT

In [13]:

def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape,
    transformer_layers,
    num_heads,
    embed_dim,
    layer_norm_eps,
    num_classes,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.3
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes)(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


## Training

In [ ]:
# Initialize model
def run_experiment():
  model = create_vivit_classifier(
      tubelet_embedder=TubeletEmbedding(
          embed_dim=cfg['PROJECTION_DIM'], patch_size=cfg['PATCH_SIZE']),
      positional_encoder=PositionalEncoder(embed_dim=cfg['PROJECTION_DIM']),
      input_shape=cfg['INPUT_SHAPE'],
      transformer_layers=cfg['NUM_LAYERS'],
      num_heads=cfg['NUM_HEADS'],
      embed_dim=cfg['PROJECTION_DIM'],
      layer_norm_eps=cfg['LAYER_NORM_EPS'],
      num_classes=cfg['NUM_CLASSES']
  )


  # Compile the model with the optimizer, loss function
  # and the metrics.
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(
      optimizer=optimizer,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[
          keras.metrics.SparseCategoricalAccuracy(name="accuracy")
      ],
  )

  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                    factor=0.1,
                                                    patience=4,
                                                    verbose=1,
                                                    cooldown=1)



  # Train the model.
  _ = model.fit(trainloader, epochs=300, validation_data=validloader, callbacks=[reduce_lr])

  _, accuracy, top_5_accuracy = model.evaluate(testloader)
  print(f"Test accuracy: {round(accuracy * 100, 2)}%")
  return model

model = run_experiment()




Epoch 1/300
1269/1269 [==============================] - 98s 67ms/step - loss: 0.6943 - accuracy: 0.5613 - val_loss: 0.7362 - val_accuracy: 0.4181 - lr: 1.0000e-04
Epoch 2/300
1269/1269 [==============================] - 85s 67ms/step - loss: 0.6819 - accuracy: 0.5766 - val_loss: 0.7158 - val_accuracy: 0.4205 - lr: 1.0000e-04
Epoch 3/300
1269/1269 [==============================] - 199s 157ms/step - loss: 0.6784 - accuracy: 0.5857 - val_loss: 0.7073 - val_accuracy: 0.4346 - lr: 1.0000e-04
Epoch 4/300
1269/1269 [==============================] - 352s 278ms/step - loss: 0.6763 - accuracy: 0.5879 - val_loss: 0.7024 - val_accuracy: 0.4559 - lr: 1.0000e-04
Epoch 5/300
1269/1269 [==============================] - 352s 278ms/step - loss: 0.6745 - accuracy: 0.5873 - val_loss: 0.6984 - val_accuracy: 0.5008 - lr: 1.0000e-04
Epoch 6/300
1269/1269 [==============================] - 352s 277ms/step - loss: 0.6729 - accuracy: 0.5888 - val_loss: 0.6952 - val_accuracy: 0.5102 - lr: 1.0000e-04
Epoch 7/

In [7]:
from tensorflow import *
from keras import *

accuracy, top_5_accuracy = model.evaluate(testloader)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

NameError: ignored

In [6]:
tf.keras.save_model('model1',model)

AttributeError: ignored